In [14]:
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
from itertools import combinations

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from library.Constants import CURRENCY_PAIRS
from library.Indicators import SMA, BB, RSI
import library.MetaTrader_Data as mtd

In [2]:
PAIRS = CURRENCY_PAIRS
TIMEFRAME = mt5.TIMEFRAME_H1
BARS = 1000
SMA_PERIODS = [14, 50]
STD_UNITS = 2
STATIONARITY_CUTOFF = 0.05
COINTEGRATION_P_VALUE_CUTOFF = 0.05

INITIAL_CAPITAL = 10000
LEVERAGE = 10
TP_SL_RATIO = 2
TRADE_SPREAD_PERCENT = 0.012
TRANSACTION_COST = 0.001
CAPITAL_ALLOCATION_PER_TRADE = 0.1

In [3]:
mtd.fill_data(PAIRS, TIMEFRAME, BARS)

Fetched data for pair: USDJPY
Fetched data for pair: EURUSD
Fetched data for pair: GBPUSD
Fetched data for pair: EURGBP
Fetched data for pair: CADCHF
Fetched data for pair: EURJPY
Fetched data for pair: AUDUSD
Fetched data for pair: USDCNH
Fetched data for pair: EURCHF
Fetched data for pair: NZDUSD


In [4]:
for symbol, df in mtd.data.items():
    for period in SMA_PERIODS:
        SMA(df, period)
        BB(df, period, STD_Units)
        RSI(df, period)

In [5]:
symbol_pairs = list(combinations(PAIRS, 2))

In [6]:
# Function to test for cointegration between two time series
def test_cointegration(data1, data2):
    coint_result = coint(data1, data2)
    return coint_result[0], coint_result[1], coint_result[2]  # Return test statistic, p-value, and critical values

def test_stationarity(series):
    adf_result = adfuller(series)
    return adf_result[1]  # Returns the p-value of ADF test

In [7]:
# Initialize an empty list to store cointegrated pairs
cointegrated_pairs = []
cointegration_results = []

# Test each pair for cointegration
for pair in symbol_pairs:
    symbol1, symbol2 = pair
    data1 = mtd.data[symbol1]['close']
    data2 = mtd.data[symbol2]['close']

    stationarity_data1_p_value = test_stationarity(data1)
    stationarity_data2_p_value = test_stationarity(data2)

    if stationarity_data1_p_value > STATIONARITY_CUTOFF and stationarity_data2_p_value > STATIONARITY_CUTOFF:  
        # Cointegration test
        test_stat, p_value, crit_values = test_cointegration(data1, data2)
        cointegration_results.append((symbol1, symbol2, p_value, test_stat, crit_values, p_value < COINTEGRATION_P_VALUE_CUTOFF))

        if p_value < COINTEGRATION_P_VALUE_CUTOFF:
            cointegrated_pairs.append(pair)
            print(f'Cointegrated Pair Found: {symbol1} and {symbol2} with p-value {p_value:.4f}')

cointegration_df = pd.DataFrame(cointegration_results, columns=['Symbol-1', 'Symbol-2', 'P-Value', 'Test Stat', 'Critical Values', 'Co-Integrated?'])
# cointegration_df

Cointegrated Pair Found: USDCNH and NZDUSD with p-value 0.0344


In [8]:
def calculate_spread(data1, data2):
    hedge_ratio = np.polyfit(data1, data2, 1)[0]  # Hedge ratio from linear regression
    spread = data1 - hedge_ratio * data2
    return spread, hedge_ratio

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Function to plot close prices, spread, and hedge ratio
def plot_spread_with_prices(data1, data2, symbol1, symbol2, spread, hedge_ratio):
    # Create subplots: 1 row, 2 cols for close prices and spread with ratio
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                        subplot_titles=(f"{symbol1} and {symbol2} Closing Prices", "Spread and Bollinger Bands"))

    # Plot Close Prices of the two symbols
    fig.add_trace(
        go.Scatter(x=data1.index, y=data1, mode='lines', name=f'{symbol1} Close', line=dict(color='blue')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=data2.index, y=data2, mode='lines', name=f'{symbol2} Close', line=dict(color='orange')),
        row=2, col=1
    )

    # Calculate Bollinger Bands for the spread
    spread_mean = spread.mean()
    spread_std = spread.std()
    upper_band = spread_mean + STD_Units * spread_std
    lower_band = spread_mean - STD_Units * spread_std

    # Plot the spread
    fig.add_trace(
        go.Scatter(x=spread.index, y=spread, mode='lines', name='Spread', line=dict(color='green')),
        row=3, col=1
    )

    # Plot Bollinger Bands for the spread
    fig.add_trace(
        go.Scatter(x=spread.index, y=[upper_band] * len(spread), mode='lines', name='Upper Band', line=dict(color='red', dash='dash')),
        row=3, col=1
    )
    fig.add_trace(
        go.Scatter(x=spread.index, y=[lower_band] * len(spread), mode='lines', name='Lower Band', line=dict(color='red', dash='dash')),
        row=3, col=1
    )

    # Add a horizontal line at the mean of the spread
    fig.add_trace(
        go.Scatter(x=spread.index, y=[spread_mean] * len(spread), mode='lines', name='Mean Spread', line=dict(color='grey', dash='dot')),
        row=3, col=1
    )

    # Update layout for aesthetics
    fig.update_layout(
        title=f"Cointegrated Pair Spread Analysis: {symbol1} vs {symbol2} - Hedge Ratio: {hedge_ratio:.4f}",
        xaxis_title='Date',
        yaxis_title='Price',
        height=800,
        template='plotly_dark'
    )

    fig.show()

# Example usage with dummy data1, data2, spread, and hedge_ratio
# Assuming calculate_spread has been run and returned 'spread' and 'hedge_ratio'

for pair in cointegrated_pairs:
    symbol1, symbol2 = pair
    data1 = mtd.data[symbol1]['close']
    data2 = mtd.data[symbol2]['close']
    
    spread, hedge_ratio = calculate_spread(data1, data2)
    plot_spread_with_prices(data1, data2, symbol1, symbol2, spread, hedge_ratio)


In [10]:
capital = INITIAL_CAPITAL
trades = []
open_positions = []

for pair in cointegrated_pairs:
    symbol1, symbol2 = pair
    data1 = mtd.data[symbol1]['close']
    data2 = mtd.data[symbol2]['close']
    
    # Calculate the spread and its bands
    spread, hedge_ratio = calculate_spread(data1, data2)
    spread_mean = spread.mean()
    spread_std = spread.std()
    upper_band = spread_mean + STD_Units * spread_std
    lower_band = spread_mean - STD_Units * spread_std

    # Loop over the price data to check for entry and exit signals
    for index, symbol1_row in mtd.data[symbol1].iterrows():
        symbol2_row = mtd.data[symbol2].loc[index]
        symbol1_price = symbol1_row['close']
        symbol2_price = symbol2_row['close']
        spread_value = spread.loc[index]
        trade_time = symbol1_row['time']
        
        # Check for entry signals (opening new positions)
        if spread_value > upper_band:
            # Short symbol1 and long symbol2
            if not any(pos['symbol'] == symbol1 and pos['position'] == 'short' for pos in open_positions) and \
               not any(pos['symbol'] == symbol2 and pos['position'] == 'long' for pos in open_positions):
                # Open short for symbol1
                open_positions.append({
                    'symbol': symbol1,
                    'entry_time': trade_time,
                    'entry_price': symbol1_price,
                    'sl': symbol1_price - (symbol1_price * TRADE_SPREAD_PERCENT),
                    'tp': symbol1_price + (symbol1_price * TRADE_SPREAD_PERCENT),
                    'position': 'short',
                    'allocated_capital': capital * CAPITAL_ALLOCATION_PER_TRADE,
                    'transaction_cost': TRANSACTION_COST * (capital * CAPITAL_ALLOCATION_PER_TRADE)
                })
                # Open long for symbol2
                open_positions.append({
                    'symbol': symbol2,
                    'entry_time': trade_time,
                    'entry_price': symbol2_price,
                    'sl': symbol2_price - (symbol2_price * TRADE_SPREAD_PERCENT),
                    'tp': symbol2_price + (symbol2_price * TRADE_SPREAD_PERCENT),
                    'position': 'long',
                    'allocated_capital': capital * CAPITAL_ALLOCATION_PER_TRADE,
                    'transaction_cost': TRANSACTION_COST * (capital * CAPITAL_ALLOCATION_PER_TRADE)
                })

        elif spread_value < lower_band:
            # Long symbol1 and short symbol2
            if not any(pos['symbol'] == symbol1 and pos['position'] == 'long' for pos in open_positions) and \
               not any(pos['symbol'] == symbol2 and pos['position'] == 'short' for pos in open_positions):
                # Open long for symbol1
                open_positions.append({
                    'symbol': symbol1,
                    'entry_time': trade_time,
                    'entry_price': symbol1_price,
                    'sl': symbol1_price + (symbol1_price * TRADE_SPREAD_PERCENT),
                    'tp': symbol1_price - (symbol1_price * TRADE_SPREAD_PERCENT),
                    'position': 'long',
                    'allocated_capital': capital * CAPITAL_ALLOCATION_PER_TRADE,
                    'transaction_cost': TRANSACTION_COST * (capital * CAPITAL_ALLOCATION_PER_TRADE)
                })
                # Open short for symbol2
                open_positions.append({
                    'symbol': symbol2,
                    'entry_time': trade_time,
                    'entry_price': symbol2_price,
                    'sl': symbol2_price + (symbol2_price * TRADE_SPREAD_PERCENT),
                    'tp': symbol2_price - (symbol2_price * TRADE_SPREAD_PERCENT),
                    'position': 'short',
                    'allocated_capital': capital * CAPITAL_ALLOCATION_PER_TRADE,
                    'transaction_cost': TRANSACTION_COST * (capital * CAPITAL_ALLOCATION_PER_TRADE)
                })

        # Check for exit signals (closing open positions)
        for position in open_positions.copy():
            df = mtd.data[position['symbol']]
            close_price = df.loc[index]['close']
            
            if position['position'] == 'long' and (close_price >= position['tp'] or close_price <= position['sl']):
                # Close long position
                profit = (close_price - position['entry_price']) * LEVERAGE * position['allocated_capital'] / position['entry_price']
                capital += profit - position['transaction_cost']
                max_drawdown = (position['entry_price'] - min(df.loc[index:, 'close'])) * LEVERAGE * position['allocated_capital'] / position['entry_price']
                
                trades.append({
                    'symbol': position['symbol'],
                    'entry_time': position['entry_time'],
                    'exit_time': trade_time,
                    'entry_price': position['entry_price'],
                    'close_price': close_price,
                    'pnl': profit - position['transaction_cost'],
                    'max_drawdown': max_drawdown,
                    'tp': position['tp'],
                    'sl': position['sl'],
                    'position': position['position']
                })
                open_positions.remove(position)
            
            elif position['position'] == 'short' and (close_price <= position['tp'] or close_price >= position['sl']):
                # Close short position
                profit = (position['entry_price'] - close_price) * LEVERAGE * position['allocated_capital'] / position['entry_price']
                capital += profit - position['transaction_cost']
                max_drawdown = (max(df.loc[index:, 'close']) - position['entry_price']) * LEVERAGE * position['allocated_capital'] / position['entry_price']
                
                trades.append({
                    'symbol': position['symbol'],
                    'entry_time': position['entry_time'],
                    'exit_time': trade_time,
                    'entry_price': position['entry_price'],
                    'close_price': close_price,
                    'pnl': profit - position['transaction_cost'],
                    'max_drawdown': max_drawdown,
                    'tp': position['tp'],
                    'sl': position['sl'],
                    'position': position['position']
                })
                open_positions.remove(position)

# Ensure all open positions are closed at the final market price
final_row = df.iloc[-1]
final_price = final_row['close']
final_time = final_row['time']

for position in open_positions:
    if position['position'] == 'long':
        profit = (final_price - position['entry_price']) * LEVERAGE * position['allocated_capital'] / position['entry_price']
    elif position['position'] == 'short':
        profit = (position['entry_price'] - final_price) * LEVERAGE * position['allocated_capital'] / position['entry_price']

    transaction_cost = TRANSACTION_COST * LEVERAGE * position['allocated_capital']
    capital += profit - transaction_cost

    max_drawdown = (position['entry_price'] - min(df['close'])) * LEVERAGE * position['allocated_capital'] / position['entry_price'] if position['position'] == 'long' else (max(df['close']) - position['entry_price']) * LEVERAGE * position['allocated_capital'] / position['entry_price']
    
    trades.append({
        'symbol': position['symbol'],
        'entry_time': position['entry_time'],
        'exit_time': final_time,
        'entry_price': position['entry_price'],
        'close_price': final_price,
        'pnl': profit - transaction_cost,
        'max_drawdown': max_drawdown,
        'tp': position['tp'],
        'sl': position['sl'],
        'position': position['position']
    })

# Convert trades to a DataFrame for analysis
trades_df = pd.DataFrame(trades)
trades_df['capital'] = INITIAL_CAPITAL + trades_df['pnl'].cumsum()

trades_df

,symbol,entry_time,exit_time,entry_price,close_price,pnl,max_drawdown,tp,sl,position,capital
0,USDCNH,2024-07-23 21:00:00,2024-07-23 21:00:00,7.29021,7.29021,-1.000000,0.315492,7.377693,7.202727,short,9999.000000
1,NZDUSD,2024-07-23 21:00:00,2024-07-29 10:00:00,0.59530,0.58811,-121.779439,146.648749,0.602444,0.588156,long,9877.220561


In [11]:
# Calculate Performance Metrics
total_profit = trades_df['pnl'].sum()
num_trades = len(trades_df)
win_rate = trades_df['pnl'].mean()
max_drawdown = trades_df['max_drawdown'].max()
avg_drawdown = trades_df['max_drawdown'].mean()
best_trade = trades_df['pnl'].max()
worst_trade = trades_df['pnl'].min()
avg_trade = trades_df['pnl'].mean()
trade_durations = (trades_df['exit_time'] - trades_df['entry_time'])
max_trade_duration = trade_durations.max()
avg_trade_duration = trade_durations.mean()
profit_factor = trades_df.loc[trades_df['pnl'] > 0, 'pnl'].sum() / -trades_df.loc[trades_df['pnl'] < 0, 'pnl'].sum()

# Capital Metrics
initial_capital = INITIAL_CAPITAL
final_capital = trades_df['capital'].iloc[-1]
max_capital = trades_df['capital'].max()
min_capital = trades_df['capital'].min()

# Annualized Volatility, Sharpe Ratio, Sortino Ratio, Calmar Ratio
trading_days = 252
annualized_volatility = trades_df['pnl'].std() * np.sqrt(trading_days)
sharpe_ratio = trades_df['pnl'].mean() / trades_df['pnl'].std() * np.sqrt(trading_days)
sortino_ratio = trades_df['pnl'].mean() / trades_df[trades_df['pnl'] < 0]['pnl'].std() * np.sqrt(trading_days)
calmar_ratio = trades_df['pnl'].sum() / max_drawdown

# Summary Table with Proper Formatting
performance_summary = pd.DataFrame({
    'Metric': ['Initial Capital', 'Final Capital', 'Maximum Capital', 'Minimum Capital', 
               'Volatility (Ann.)', 'Sharpe Ratio', 'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown',
               'Avg. Drawdown', '# Trades', 'Win Rate', 'Best Trade', 'Worst Trade', 'Avg. Trade',
               'Max. Trade Duration', 'Avg. Trade Duration', 'Profit Factor'],
    'Value': [initial_capital, final_capital, max_capital, min_capital,
              annualized_volatility, sharpe_ratio, sortino_ratio, calmar_ratio, max_drawdown,
              avg_drawdown, num_trades, win_rate, best_trade, worst_trade, avg_trade,
              max_trade_duration, avg_trade_duration, profit_factor]
})

# Apply formatting to the 'Value' column
performance_summary['Value'] = performance_summary['Value'].apply(lambda x: f'{x:.2f}' if isinstance(x, (int, float)) else x)

# Display performance summary
performance_summary

,Metric,Value
0,Initial Capital,10000.00
1,Final Capital,9877.22
2,Maximum Capital,9999.00
3,Minimum Capital,9877.22
4,Volatility (Ann.),1355.75
5,Sharpe Ratio,-11.41
6,Sortino Ratio,-11.41
7,Calmar Ratio,-0.84
8,Max. Drawdown,146.65
9,Avg. Drawdown,73.48


In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define colors for better visibility
entry_color = '#2ca02c'  # Green for entry
exit_color = '#d62728'   # Red for exit
long_color = '#1f77b4'   # Blue for long positions
short_color = '#ff7f0e'  # Orange for short positions

def plot_spread_with_prices(pair):
    symbol1, symbol2 = pair
    df1 = mtd.data[symbol1]
    df2 = mtd.data[symbol2]
    
    spread, hedge_ratio = calculate_spread(df1['close'], df2['close'])

    # Create subplots: 3 rows, 1 column for close prices and spread
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                        subplot_titles=[f'{symbol1} Close', f'{symbol2} Close', 'Spread'],
                        vertical_spacing=0.1)

    # Plot Close Prices of the two symbols
    fig.add_trace(
        go.Scatter(x=df1['time'], y=df1['close'], mode='lines', name=f'{symbol1} Close', line=dict(color='blue')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=df2['time'], y=df2['close'], mode='lines', name=f'{symbol2} Close', line=dict(color='orange')),
        row=2, col=1
    )

    # Plot the spread and Bollinger Bands
    spread_mean = spread.mean()
    spread_std = spread.std()
    upper_band = spread_mean + STD_Units * spread_std
    lower_band = spread_mean - STD_Units * spread_std

    fig.add_trace(
        go.Scatter(x=df1['time'], y=spread, mode='lines', name='Spread', line=dict(color='green')),
        row=3, col=1
    )
    fig.add_trace(
        go.Scatter(x=df1['time'], y=[upper_band] * len(spread), mode='lines', name='Upper Band', line=dict(color='red', dash='dash')),
        row=3, col=1
    )
    fig.add_trace(
        go.Scatter(x=df1['time'], y=[lower_band] * len(spread), mode='lines', name='Lower Band', line=dict(color='red', dash='dash')),
        row=3, col=1
    )
    fig.add_trace(
        go.Scatter(x=df1['time'], y=[spread_mean] * len(spread), mode='lines', name='Mean Spread', line=dict(color='grey', dash='dot')),
        row=3, col=1
    )

    # Plot trades
    for index, trade in trades_df.iterrows():
        if trade['symbol'] == symbol1:
            row = 1
        elif trade['symbol'] == symbol2:
            row = 2
        else:
            continue  # Skip if the trade doesn't belong to either symbol

        color = long_color if trade['position'] == 'long' else short_color
        
        # Entry point
        fig.add_trace(
            go.Scatter(x=[trade['entry_time']], y=[trade['entry_price']], mode='markers', 
                       marker=dict(color=entry_color, size=8, symbol='triangle-up'), 
                       name='Entry', showlegend=False),
            row=row, col=1
        )
        
        # Exit point
        fig.add_trace(
            go.Scatter(x=[trade['exit_time']], y=[trade['close_price']], mode='markers', 
                       marker=dict(color=exit_color, size=8, symbol='triangle-down'), 
                       name='Exit', showlegend=False),
            row=row, col=1
        )
        
        # Trade line
        fig.add_trace(
            go.Scatter(x=[trade['entry_time'], trade['exit_time']], 
                       y=[trade['entry_price'], trade['close_price']], 
                       mode='lines', line=dict(color=color, width=2),
                       name=f"{trade['position'].capitalize()} Trade", showlegend=False),
            row=row, col=1
        )

    # Update layout for aesthetics
    fig.update_layout(
        title=f"Cointegrated Pair Analysis: {symbol1} vs {symbol2} - Hedge Ratio: {hedge_ratio:.4f}",
        xaxis_title='Date',
        height=1000,
        template='plotly_dark',
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    # Update y-axis titles
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Price", row=2, col=1)
    fig.update_yaxes(title_text="Spread", row=3, col=1)

    fig.show()

# Plot for each cointegrated pair
for pair in cointegrated_pairs:
    plot_spread_with_prices(pair)